In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
import torch.nn.functional as F

In [2]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

# Hyperparameters
num_epochs = 5
num_classes = 10
batch_size = 64
learning_rate = 0.001

In [3]:
# Load Data
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
                                
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True, 
                                             download=True,
                                             transform=transform)
                                             
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)
                                
test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=False,
                                             transform=transform)
                                             
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

Files already downloaded and verified


In [4]:
# ConvNet Model
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*8*8, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 32*8*8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet(num_classes).to(device)

In [5]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Train Network
train_loss = 0.0
train_acc = 0.0

for epoch in range(num_epochs):
    
    train_loss = 0.0
    train_acc = 0.0
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()

    # Print metrics
    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    print(f'Training Loss: {train_loss:.4f}  |  Training Accuracy: {train_acc:.4f}')

    # Save checkpoint
    torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              }, 'checkpoints/checkpoint.pt')

Epoch 1/5
Training Loss: 0.0204  |  Training Accuracy: 0.5324
Epoch 2/5
Training Loss: 0.0152  |  Training Accuracy: 0.6545
Epoch 3/5
Training Loss: 0.0134  |  Training Accuracy: 0.6969
Epoch 4/5
Training Loss: 0.0121  |  Training Accuracy: 0.7264
Epoch 5/5
Training Loss: 0.0110  |  Training Accuracy: 0.7519


In [7]:
# Evaluation
val_loss = 0.0
val_acc = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        # Forward pass
        val_loss += criterion(outputs, labels)
        val_acc += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(test_loader.dataset)
    val_acc /= len(test_loader.dataset)
    
    print(f' Validation Loss: {val_loss:.4f}  |  Validation Accuracy: {val_acc:.4f}')

 Validation Loss: 0.0137  |  Validation Accuracy: 0.6973


In [8]:
# Save model
torch.save(model, 'models/cifar10.pt')

In [9]:
# Load model
model = torch.load('models/cifar10.pt')

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Load checkpoint  
checkpoint = torch.load('checkpoints/checkpoint.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

In [11]:
# Continue training
num_epochs = 10
train_loss = 0.0
train_acc = 0.0

for epoch in range(epoch, num_epochs):
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += (outputs.argmax(1) == labels).sum().item()

    # Print metrics
    train_loss /= len(train_loader.dataset)
    train_acc /= len(train_loader.dataset)

    print(f'Training Loss: {train_loss:.4f}  |  Training Accuracy: {train_acc:.4f}')

Epoch 5/10
Training Loss: 0.0100  |  Training Accuracy: 0.7744
Epoch 6/10
Training Loss: 0.0090  |  Training Accuracy: 0.7955
Epoch 7/10
Training Loss: 0.0083  |  Training Accuracy: 0.8117
Epoch 8/10
Training Loss: 0.0074  |  Training Accuracy: 0.8305
Epoch 9/10
Training Loss: 0.0068  |  Training Accuracy: 0.8453
Epoch 10/10
Training Loss: 0.0061  |  Training Accuracy: 0.8599


In [12]:
# Evaluation
val_loss = 0.0
val_acc = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        # Forward pass
        val_loss += criterion(outputs, labels)
        val_acc += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(test_loader.dataset)
    val_acc /= len(test_loader.dataset)
    
    print(f' Validation Loss: {val_loss:.4f}  |  Validation Accuracy: {val_acc:.4f}')

 Validation Loss: 0.0155  |  Validation Accuracy: 0.7089
